In [2]:
# Imports

import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import os

cwd = os.getcwd()
print(cwd)


/Users/jakobsommer/Library/CloudStorage/OneDrive-UniklinikRWTHAachen/Dokumente/ADRED_automatic_v3/pipeline_v2


In [3]:
drc = pd.read_json(r'drc_p3eva.json', orient='index', convert_axes=False, dtype=False)
drc.drop(columns=['3a4sub', '1a2sub'], inplace=True)
drc


,case_id,Z_Wirkstoff_Gesamt,Z_ATC_Gesamt,AllInteraction,2d6sub,2c19sub,2c9sub
10_1_1,10_1,Phenprocoumon,B01AA04,"[3a4sub, 2c9sub]",0,0,1
10_1_2,10_1,Oxcarbazepin,N03AF02,"[2c19inh, 3a4ind, 3a4inh]",0,0,0
10_1_3,10_1,Pravastatin,C10AA03,[],0,0,0
10_1_4,10_1,Levetiracetam,N03AX14,[],0,0,0
10_1_5,10_1,Pantoprazol,A02BC02,"[2c19inh, 2c19sub, 3a4sub]",0,1,0
...,...,...,...,...,...,...,...
999_1_5,999_1,Trospium,G04BD09,[2d6inh],0,0,0
999_1_6,999_1,Metoprolol,C07AB02,"[3a4sub, 2d6sub, 2d6inh]",1,0,0
999_1_7,999_1,Cabergolin,N04BC06,[3a4sub],0,0,0
999_1_8,999_1,Ramipril,C09AA05,[],0,0,0


Here we took out the file that only had classification from <b>ATC alone</b>:

In [4]:
# Import for automatic classification (ATC only)
drcatc = pd.read_json(r'drc_p1_pv2.json', orient='index', convert_axes=False, dtype=False)
drcatc

,case_id,Z_Wirkstoff_Gesamt,Z_ATC_Gesamt,AllInteraction
10_1_1,10_1,Phenprocoumon,B01AA04,"[3a4sub, 2c9sub]"
10_1_2,10_1,Oxcarbazepin,N03AF02,"[2c19inh, 3a4ind, 3a4inh]"
10_1_3,10_1,Pravastatin,C10AA03,[]
10_1_4,10_1,Levetiracetam,N03AX14,[]
10_1_5,10_1,Pantoprazol,A02BC02,"[2c19inh, 2c19sub, 3a4sub]"
...,...,...,...,...
999_1_5,999_1,Trospium,G04BD09,[2d6inh]
999_1_6,999_1,Metoprolol,C07AB02,"[3a4sub, 2d6sub, 2d6inh]"
999_1_7,999_1,Cabergolin,N04BC06,[3a4sub]
999_1_8,999_1,Ramipril,C09AA05,[]


In [5]:

man = pd.read_excel(r'CYP_Substrate_haendisch.xlsx')
man = man.loc[:,['case_line_id', 'CYP2C9.1', 'CYP2C19.1', 'CYP2D6.1']]
man = man.set_index('case_line_id')
man

,CYP2C9.1,CYP2C19.1,CYP2D6.1
case_line_id,,,
10_1_1,1.0,0.0,0.0
10_1_2,NaN,NaN,NaN
10_1_3,NaN,NaN,NaN
10_1_4,NaN,NaN,NaN
10_1_5,0.0,1.0,0.0
...,...,...,...
999_1_5,NaN,NaN,NaN
999_1_6,0.0,0.0,1.0
999_1_7,NaN,NaN,NaN


<h2>Pre-Analysis Modification</h2>
<br>
Mark all data points of automatic classification with err1 as np.nan to increase compability with manual classfication. Save in a mastersheet called data. Further analysis has to be done on the mastersheet. 

In [6]:
# mark all those who could not been found as np.nan
s = drc.AllInteraction.str.contains('err1', regex=False)
err_drc = drc.loc[s].copy()
err_drc.loc[:, '2d6sub':'2c9sub'] = 99 #um nan = 0 nicht mit kein-substrat=0 zu verwechseln
drc.update(err_drc)
drc.replace(99, np.nan, inplace=True) 
# ab jetzt alle fehlende Daten, die zuvor mit 'err1' markiert sind als np.nan

del s

mastersheet = drc.join(man, how='inner', lsuffix='_drc', rsuffix='_man')
mastersheet

,case_id,Z_Wirkstoff_Gesamt,Z_ATC_Gesamt,AllInteraction,2d6sub,2c19sub,2c9sub,CYP2C9.1,CYP2C19.1,CYP2D6.1
10_1_1,10_1,Phenprocoumon,B01AA04,"[3a4sub, 2c9sub]",0.0,0.0,1.0,1.0,0.0,0.0
10_1_2,10_1,Oxcarbazepin,N03AF02,"[2c19inh, 3a4ind, 3a4inh]",0.0,0.0,0.0,NaN,NaN,NaN
10_1_3,10_1,Pravastatin,C10AA03,[],0.0,0.0,0.0,NaN,NaN,NaN
10_1_4,10_1,Levetiracetam,N03AX14,[],0.0,0.0,0.0,NaN,NaN,NaN
10_1_5,10_1,Pantoprazol,A02BC02,"[2c19inh, 2c19sub, 3a4sub]",0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
999_1_5,999_1,Trospium,G04BD09,[2d6inh],0.0,0.0,0.0,NaN,NaN,NaN
999_1_6,999_1,Metoprolol,C07AB02,"[3a4sub, 2d6sub, 2d6inh]",1.0,0.0,0.0,0.0,0.0,1.0
999_1_7,999_1,Cabergolin,N04BC06,[3a4sub],0.0,0.0,0.0,NaN,NaN,NaN
999_1_8,999_1,Ramipril,C09AA05,[],0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Create pair to make it easier for later analysis
pair = {
    '2d6': ['2d6sub', 'CYP2D6.1'],
    '2c9': ['2c9sub', 'CYP2C9.1'],
    '2c19': ['2c19sub', 'CYP2C19.1']
}

<h2>Classification rates</h2>

In [8]:
pd.isna(mastersheet).sum()

case_id                  0
Z_Wirkstoff_Gesamt       0
Z_ATC_Gesamt             5
AllInteraction           0
2d6sub                 983
2c19sub                983
2c9sub                 983
CYP2C9.1              8199
CYP2C19.1             8199
CYP2D6.1              8199
dtype: int64

<h3>By entries</h3>
How is the classification rate when we look at the entries?

In [9]:
total_entr = len(mastersheet)

amount_class = total_entr - pd.isna(mastersheet['CYP2C9.1']).sum()
perc_class = amount_class/total_entr

amount_unclass = pd.isna(mastersheet['CYP2C9.1']).sum()
perc_unclass = amount_unclass/total_entr

print(f'Manual classification - by entries:\n', 
    f'Total entries: {total_entr}\n', 
    f'Classified: {amount_class} | {perc_class}\n',
    f'Unclassified: {amount_unclass} | {perc_unclass}')
del total_entr, amount_class, perc_class, amount_unclass, perc_unclass

Manual classification - by entries:
 Total entries: 21721
 Classified: 13522 | 0.6225311910133051
 Unclassified: 8199 | 0.3774688089866949


In [10]:
total_entr = len(drcatc)

s = drcatc.AllInteraction.str.contains('err1', regex=False)
drcatc_err = drcatc.loc[s]
drcatc_errno = drcatc.loc[~s]

amount_class = len(drcatc_errno)
perc_class = amount_class/total_entr

amount_unclass = len(drcatc_err)
perc_unclass = amount_unclass/total_entr

print(f'Automatic classification - by entries (ATC only):\n', 
    f'Total entries: {total_entr}\n', 
    f'Classified: {amount_class} | {perc_class}\n',
    f'Unclassified: {amount_unclass} | {perc_unclass}')
del total_entr, amount_class, perc_class, amount_unclass, perc_unclass
del drcatc_err, drcatc_errno, s

Automatic classification - by entries (ATC only):
 Total entries: 21721
 Classified: 19746 | 0.9090741678559919
 Unclassified: 1975 | 0.09092583214400811


In [11]:
total_entr = len(mastersheet)

amount_class = total_entr - pd.isna(mastersheet['2d6sub']).sum()
perc_class = amount_class/total_entr

amount_unclass = pd.isna(mastersheet['2d6sub']).sum()
perc_unclass = amount_unclass/total_entr

print(f'Automatic classification - by entries (ATC and substance name):\n', 
    f'Total entries: {total_entr}\n', 
    f'Classified: {amount_class} | {perc_class}\n',
    f'Unclassified: {amount_unclass} | {perc_unclass}')
del total_entr, amount_class, perc_class, amount_unclass, perc_unclass

Automatic classification - by entries (ATC and substance name):
 Total entries: 21721
 Classified: 20738 | 0.9547442567100962
 Unclassified: 983 | 0.04525574328990378


<h3>By substances</h3>
How is the classification rate when we look at the substances instead of entries?

In [12]:
# Extract from mastersheet
man_mod = mastersheet.loc[:,["Z_Wirkstoff_Gesamt",'CYP2C9.1']].copy()

man_err = man_mod.loc[np.isnan(man_mod['CYP2C9.1'])]
man_err_gr = man_err.groupby(by=["Z_Wirkstoff_Gesamt"]).size()

man_errno = man_mod.loc[~np.isnan(man_mod['CYP2C9.1'])]
man_errno_gr = man_errno.groupby(by=["Z_Wirkstoff_Gesamt"]).size() #110

total_entr = len(man_mod.groupby(by=["Z_Wirkstoff_Gesamt"]).size())

amount_class = len(man_errno_gr)
perc_class = amount_class / total_entr

amount_unclass = len(man_err_gr)
perc_unclass = amount_unclass / total_entr

print(f'Manual classification:\n', 
    f'Total entries: {total_entr}\n', 
    f'Classified: {amount_class} | {perc_class}\n',
    f'Unclassified: {amount_unclass} | {perc_unclass}')
del total_entr, amount_class, perc_class, amount_unclass, perc_unclass
del man_mod, man_err, man_err_gr, man_errno, man_errno_gr

Manual classification:
 Total entries: 995
 Classified: 110 | 0.11055276381909548
 Unclassified: 885 | 0.8894472361809045


In [13]:
m_mod = drcatc.copy()
s = m_mod.AllInteraction.str.contains('err1', regex=False)
m_mod_err = m_mod.loc[s]
m_mod_err_gr = m_mod_err.groupby(by=["Z_Wirkstoff_Gesamt"]).size()
m_mod_errno = m_mod.loc[~s]
m_mod_errno_gr = m_mod_errno.groupby(by=["Z_Wirkstoff_Gesamt"]).size()

total_entr = len(m_mod.groupby(by=["Z_Wirkstoff_Gesamt"]).size())

amount_class = len(m_mod_errno_gr)
perc_class = amount_class / total_entr

amount_unclass = len(m_mod_err_gr)
perc_unclass = amount_unclass / total_entr

print(f'Automatic classification - by substances (ATC only):\n', 
    f'Total entries: {total_entr}\n', 
    f'Classified: {amount_class} | {perc_class}\n',
    f'Unclassified: {amount_unclass} | {perc_unclass}\n', 
    f'Classified (calculated): {total_entr-amount_unclass} | {1.00-perc_unclass}')

del s, m_mod, m_mod_err, m_mod_err_gr, m_mod_errno, m_mod_errno_gr

Automatic classification - by substances (ATC only):
 Total entries: 995
 Classified: 703 | 0.7065326633165829
 Unclassified: 349 | 0.3507537688442211
 Classified (calculated): 646 | 0.649246231155779


In [14]:
s = drc.AllInteraction.str.contains('err1', regex=False)
drc_err = drc.loc[s]
drc_err_gr = drc_err.groupby(by=["Z_Wirkstoff_Gesamt"]).size()
drc_errno = drc.loc[~s]
drc_errno_gr = drc_errno.groupby(by=["Z_Wirkstoff_Gesamt"]).size()

total_entr = len(drc.groupby(by=["Z_Wirkstoff_Gesamt"]).size())

amount_class = len(drc_errno_gr)
perc_class = amount_class / total_entr

amount_unclass = len(drc_err_gr)
perc_unclass = amount_unclass / total_entr

print(f'Automatic classification - by substances (ATC and substance name):\n', 
    f'Total entries: {total_entr}\n', 
    f'Classified: {amount_class} | {perc_class}\n',
    f'Unclassified: {amount_unclass} | {perc_unclass}\n',
    f'Classified (calculated): {total_entr-amount_unclass} | {1.00-perc_unclass}')
del total_entr, amount_class, perc_class, amount_unclass, perc_unclass
del s, drc_err

Automatic classification - by substances (ATC and substance name):
 Total entries: 995
 Classified: 772 | 0.7758793969849246
 Unclassified: 238 | 0.23919597989949748
 Classified (calculated): 757 | 0.7608040201005025


The numbers from the automatic classification did not add up to 100%, reason is that there are substances that have been both classified and unclassified. The reason is probably that the same substance has different ATC codes. Some of them could be found while others not and the substance could not been identified by the substance name. <br>
Therefore added Classified calculated.<br>
<br>
Here it can be seen, where the difference between Classified and Classified (calculated) comes from:

In [15]:
a = set(drc_err_gr.index)
b = set(drc_errno_gr.index)
overlap = list(a & b)
print(overlap)
print(len(overlap))
del a,b

['Insulin-Isophan (human)', 'Macrogol', 'Ipratropiumbromid', 'Hyaluronsäure', 'Insulin (human)', 'Calciumfolinat', 'Leuprorelin', 'Beclometason', 'Aciclovir', 'Eisen(II)glycinsulfat', 'Acetylsalicylsäure', 'Ciclosporin', 'Cineol', 'Natriumchlorid', 'Kaliumchlorid']
15


<h2>NaN Differences</h2>
Analyze the difference between the NaNs:

In [16]:
# Check enzyme 

# 0 if you want to test the automatic method
# 1 if you want to test the manual method


def compare_nan(enzyme, whosperformance):
    data = mastersheet.copy()
    # Drop all na from the method that you do not want to test
    if whosperformance == 0: 
        # I do not want to test the manual method
        data.dropna(subset=['CYP2C19.1', 'CYP2C9.1', 'CYP2D6.1'], inplace=True)
    elif (whosperformance == 1): 
        # I do not want to test the automatic method
        data.dropna(subset=['2d6sub', '2c19sub', '2c9sub'], inplace=True)
    # For the method I want to test, check which NaNs are left, 
    # these are the one that the other method managed to find
    isnamedis = data.loc[pd.isna(data[enzyme])]
    table = isnamedis['Z_Wirkstoff_Gesamt'].value_counts()
    return table

def compare_nan_pairwise(li):
    # NaNs that the Computer produced (but hand does not)
    li1 = compare_nan(li[0], 0)
    # NaNs that the Hand produced (but the Computer does not)
    li2 = compare_nan(li[1], 1)
    return li1, li2



All the enzymes can be put into the function, the first focuses on the limitations of the computer, the second table on the limitations of the manual method

In [17]:
print(pair)

# It does not matter which enzyme to pick 
table_nan1, table_nan2 = compare_nan_pairwise(pair['2c9'])

total_subs = len(mastersheet.groupby(by=["Z_Wirkstoff_Gesamt"]).size())
total_entr = len(mastersheet)

table_nan1 = table_nan1.to_frame()
table_nan1['portion_entr'] = table_nan1['Z_Wirkstoff_Gesamt'] / total_entr
table_nan1['portion_subs'] = table_nan1['Z_Wirkstoff_Gesamt'] / total_subs

table_nan2 = table_nan2.to_frame()
table_nan2['portion_entr'] = table_nan2['Z_Wirkstoff_Gesamt'] / total_entr
table_nan2['portion_subs'] = table_nan2['Z_Wirkstoff_Gesamt'] / total_subs

print(table_nan1.sum())
print(table_nan2.sum())

{'2d6': ['2d6sub', 'CYP2D6.1'], '2c9': ['2c9sub', 'CYP2C9.1'], '2c19': ['2c19sub', 'CYP2C19.1']}
Z_Wirkstoff_Gesamt    154.000000
portion_entr            0.007090
portion_subs            0.154774
dtype: float64
Z_Wirkstoff_Gesamt    7370.000000
portion_entr             0.339303
portion_subs             7.407035
dtype: float64


<h2>Compare classification differences</h2>

Now, delete all unclassified entries / substances and focus on the classified ones. See how many entries are left and calculate the differences. 

In [18]:
data = mastersheet.copy()

data.dropna(subset=['2d6sub', '2c19sub', '2c9sub', 'CYP2D6.1', 'CYP2C19.1', 'CYP2C9.1'], inplace=True)
print("Number of entries: ", len(data))
print("Number of substances: ", len(data.groupby(by=["Z_Wirkstoff_Gesamt"]).size()))




Number of entries:  13368
Number of substances:  91


Change Bisoprolol to 0, since we have updated our manual bisoprolol classification

In [19]:
s = data.Z_Wirkstoff_Gesamt.str.contains('Bisoprolol')
data.loc[s, 'CYP2D6.1'] = 0.000
data = data.copy()
del s

<h3>Compare portion ob substrates by entry</h3>

In [20]:

# Wie viele Positives gibt es im Datenset?
print(data.iloc[:,4:].sum())
print()
print(data.iloc[:,4:].sum() / len(data))

2d6sub       2509.0
2c19sub      2906.0
2c9sub       3511.0
CYP2C9.1     1737.0
CYP2C19.1    2010.0
CYP2D6.1     2507.0
dtype: float64

2d6sub       0.187687
2c19sub      0.217385
2c9sub       0.262642
CYP2C9.1     0.129937
CYP2C19.1    0.150359
CYP2D6.1     0.187537
dtype: float64


<h3>Compare portion on substrates by substance</h3>

Group all entries into substances and use one hot encoding to see if this substance is a substrate or not. 

In [21]:
data_med = data.groupby(by=['Z_Wirkstoff_Gesamt']).sum()
data_med[data_med > 0] = 1
print(data_med.sum())
print()
print(data_med.sum() / len(data_med))

2d6sub       17.0
2c19sub      15.0
2c9sub       24.0
CYP2C9.1      9.0
CYP2C19.1     9.0
CYP2D6.1     15.0
dtype: float64

2d6sub       0.186813
2c19sub      0.164835
2c9sub       0.263736
CYP2C9.1     0.098901
CYP2C19.1    0.098901
CYP2D6.1     0.164835
dtype: float64


<h3>Show classifcation differences</h3>

In [22]:
print("\nCohen's Kappa Score - by entries: 2d6, 2c19, 2c9")
print(cohen_kappa_score(data['2d6sub'], data['CYP2D6.1']))
print(cohen_kappa_score(data['2c19sub'], data['CYP2C19.1']))
print(cohen_kappa_score(data['2c9sub'], data['CYP2C9.1']))


Cohen's Kappa Score - by entries: 2d6, 2c19, 2c9
0.9995091949037778
0.7783337634251953
0.5908276417167244


In [23]:
print("\nCohen's Kappa Score - by substances: 2d6, 2c19, 2c9")
print(cohen_kappa_score(data_med['2d6sub'], data_med['CYP2D6.1']))
print(cohen_kappa_score(data_med['2c19sub'], data_med['CYP2C19.1']))
print(cohen_kappa_score(data_med['2c9sub'], data_med['CYP2C9.1']))


Cohen's Kappa Score - by substances: 2d6, 2c19, 2c9
0.9242298084929226
0.7147335423197492
0.46907817969661614


In [24]:
def find_diff(li):
    opt = data[['Z_Wirkstoff_Gesamt',li[0], li[1]]]
    optdiff = opt.loc[opt.loc[:,li[0]] != opt.loc[:,li[1]]]
    optgb = optdiff.groupby(by=["Z_Wirkstoff_Gesamt"]).sum()
    return optgb

In [25]:
#table = find_diff(pair['2d6'])
table = find_diff(pair['2c9'])